## Review: what we did in Week 3: Amazon data
* Read Amazon.csv
* Get to know the data
* Create a smaller subset of the data
## [Jump to Week 4 material](#thisWeek)

In [1]:
# imports and specifications
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### read Amazon.csv

In [2]:
#amazon = pd.read_csv('/Users/Aaron_hill/Dropbox/data/Amazon.csv')
amazon = pd.read_csv('/Users/aaron/Dropbox/data/Amazon.csv')

### get to know the data

In [3]:
print("amazon is:", type(amazon))
print("amazon has", amazon.shape[0], "rows and", amazon.shape[1], "columns", "\n")
print("the data types for each of the columns in amazon:")
print(amazon.dtypes, "\n")
print("the first 10 rows in amazon:")
print(amazon.head(10))

amazon is: <class 'pandas.core.frame.DataFrame'>
amazon has 455000 rows and 13 columns 

the data types for each of the columns in amazon:
Unnamed: 0                  int64
Id                          int64
ProductId                  object
UserId                     object
ProfileName                object
HelpfulnessNumerator        int64
HelpfulnessDenominator      int64
Score                       int64
Time                        int64
Summary                    object
Text                       object
helpScore                 float64
helpful                      bool
dtype: object 

the first 10 rows in amazon:
   Unnamed: 0      Id   ProductId          UserId       ProfileName  \
0      138806  138807  B000E63LME  A1CQGW1AOD0LF2  Alena K. "Alena"   
1      469680  469681  B004ZIH4KM  A37S7U1OX2MCWI        Becky Cole   
2      238202  238203  B003ZXE9QA  A2OM6G73E64EQ9              jeff   
3      485307  485308  B001RVFERK  A25W349EE97NBK          Tangent4   
4      375283  3752

### create a smaller subset of the data

In [4]:
# create a subset of "amazon" that contains all the columns but only only the first 1000 rows
amazon_subset = amazon[:1000]
print(type(amazon_subset))
print(amazon_subset.shape)

<class 'pandas.core.frame.DataFrame'>
(1000, 13)


### create a ndarray for `L`

In [5]:
L = amazon_subset["helpful"]
print(type(L))
print(type(L.values))
print(L.shape)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
(1000,)


### create a ndarray for `X`
Use only "Score" and "Time" as features, for now.

In [6]:
X = amazon_subset[["Score", "Time"]]
print(type(X))
print(type(X.values))
print(X.shape)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(1000, 2)


### using the `SVC` class in `sklearn.svm`, fit the SVM model according to the given training data
For now, accept all the default parameters in `SVC`

In [7]:
from sklearn.svm import SVC
clf = SVC() # accepting all the default parameters
clf.fit(X, L)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### how well did we do? compare the model's predictions for  `Y` to the labels `L`

In [8]:
# number of accurate predictions
print(sum(clf.predict(X) == L.values))

986


## <a name='thisWeek'></a>Week 4: fit SVM using gradient descent and assess the fit of the model

### using the `SGDClassifier` class in `linear_model`, fit the model according to given training data

In [9]:
from sklearn import linear_model
sgd = linear_model.SGDClassifier()
sgd.fit(X, L)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False)

### how well did we do? compare the model's predictions for  `Y` to the labels `L`

In [10]:
class BinaryClassificationPerformance():
    '''Performance measures to evaluate the fit of a binary classification model'''
    
    def __init__(self, predictions, labels, probabilities=None):
        '''Initialize attributes: predictions-vector of predicted values for Y, labels-vector of labels for Y'''
        '''probabilities-optional, probability that Y is equal to True'''
        self.probabilities = probabilities
        self.performance_df = pd.concat([pd.DataFrame(predictions), pd.DataFrame(labels)], axis=1)
        self.performance_df.columns = ['preds', 'labls']
        self.performance_measures = {}
  
    def compute_measures(self):
        '''Compute performance measures defined by Flach p. 57'''
        self.performance_measures['Pos'] = self.performance_df['preds'].sum()
        self.performance_measures['Neg'] = self.performance_df.shape[0] - self.performance_df['preds'].sum()
        self.performance_measures['TP'] = ((self.performance_df['preds'] == True) & (self.performance_df['labls'] == True)).sum()
        self.performance_measures['TN'] = ((self.performance_df['preds'] == False) & (self.performance_df['labls'] == False)).sum()
        self.performance_measures['FP'] = ((self.performance_df['preds'] == True) & (self.performance_df['labls'] == False)).sum()
        self.performance_measures['FN'] = ((self.performance_df['preds'] == False) & (self.performance_df['labls'] == True)).sum()

In [11]:
tc = BinaryClassificationPerformance(sgd.predict(X), L.values)
tc.compute_measures()
print(tc.performance_measures)

{'FN': 63, 'TP': 0, 'Pos': 0, 'TN': 937, 'Neg': 1000, 'FP': 0}
